# ***PROJECT PLAN SUMMARY***

To forecast real estate prices and identify the top 5 best zip codes for investment using the Zillow Research dataset, we'll go through a series of steps involving data preprocessing and analysis. The process will include:

- ***Data Loading and Inspection:*** We'll start by loading the dataset and inspecting its structure to understand the type of data it contains, including the number of records, columns, and initial observations about the data quality (missing values, data types, etc.).

- ***Converting to Long Format:*** Since the data is in wide format (where each column represents a different time point), we'll reshape it into a long format. In long format, each row represents a single time point for a particular zip code. This is essential for time series analysis and modeling.

- ***Data Cleaning:*** This step will involve handling missing values, outliers, and any anomalies in the data. 

- ***Feature Engineering:*** We might need to create additional features that could be important for the analysis. This can include calculating metrics like year-over-year price growth, average prices, etc.

- ***Exploratory Data Analysis (EDA):*** We'll conduct EDA to understand trends, patterns, and relationships within the data. This step is crucial for gaining insights and guiding the modeling process.

- ***Time Series Forecasting Model:*** We'll select and apply suitable time series forecasting models (like ARIMA, SARIMA, Prophet, etc.) to predict future real estate prices for each zip code.

- ***Evaluation and Selection:*** Using forecast results and possibly other economic indicators, we'll evaluate and rank the zip codes based on investment potential. Criteria might include forecasted price appreciation, stability of the market, etc.

- ***Reporting:*** Finally, we'll compile our findings and recommendations into a report for the investment firm, highlighting the top 5 zip codes for investment along with the rationale for each selection.


------------------------------------

# ***1.BUSINESS UNDERSTANDING***
## ***Goal:***

- Identify the top 5 zip codes for real estate investment, considering potential return on investment, market stability, and future growth prospects.

## ***Key Variables (Model Targets):***
- Price Appreciation: Growth in property values over time.
- Market Stability: Consistency in price trends, indicating lower risk.
- Demand Indicators: Factors influencing the desirability of the area (e.g., demographics, economic growth).
- Investment Return Potential: Estimated return based on historical and forecasted data.

## ***Data Source Identification:***

-Primary Source: Zillow Research dataset, providing historical real estate prices by zip code.

## ***Objectives:***
#### ***Project Goals:***
- Quantitative Analysis: Analyze historical price trends and forecast future growth.

### ***Success Metrics:***
- Financial Returns: price appreciation percentage = atleast 20%
- Risk Assessment: Evaluate and limit the investment risk based on market stability.

----------------------
# ***2.DATA ACQUISATION AND UNDERSTANDING***

The Zillow dataset provides detailed real estate data, with each row representing a unique zip code. Here's an overview of the dataset structure:

- RegionID: A unique identifier for each region.
- RegionName: The zip code for the region.
- City: The city where the region is located.
- State: The state where the region is located.
- Metro: The metropolitan area associated with the region.
- CountyName: The name of the county where the region is located.
- SizeRank: A ranking of the region based on size.
- Monthly Price Data: Starting from April 1996 to April 2018, this dataset includes monthly real estate prices for each zip code.

We'll analyze historical price trends at a zip code level, which is crucial for our objective of identifying the top 5 zip codes for real estate investment. The analysis will involve:

- Trend Analysis: Evaluating the long-term price trends in each zip code.
- Volatility Assessment: Understanding the stability or variability in prices over time.
- Comparative Analysis: Comparing zip codes across different regions, cities, or states.
- Forecasting: Applying statistical or machine learning models to predict future price trends.


In [ ]:
#Necessary imports
import pandas as pd

In [4]:
#Data Loading and Inspection
# Loading the dataset
zillow_data = pd.read_csv('data/zillow_data.csv')
zillow_data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [3]:
# Reshaping the data to long format
zillow_long = pd.melt(zillow_data, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'],
                      var_name='Date', value_name='Price')

# Convert the 'Date' column to a datetime type
zillow_long['Date'] = pd.to_datetime(zillow_long['Date'])

# Check the first few rows of the reshaped data and its data types
reshaped_data = zillow_long
data_types = zillow_long.dtypes
missing_values = zillow_long.isnull().sum()

reshaped_data 


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Date,Price
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,1996-04-01,235700.0
2,91982,77494,Katy,TX,Houston,Harris,3,1996-04-01,210400.0
3,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
4,93144,79936,El Paso,TX,El Paso,El Paso,5,1996-04-01,77300.0
...,...,...,...,...,...,...,...,...,...
3901590,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,2018-04-01,209300.0
3901591,59107,3293,Woodstock,NH,Claremont,Grafton,14720,2018-04-01,225800.0
3901592,75672,40404,Berea,KY,Richmond,Madison,14721,2018-04-01,133400.0
3901593,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,2018-04-01,664400.0


In [4]:
data_types

RegionID               int64
RegionName             int64
City                  object
State                 object
Metro                 object
CountyName            object
SizeRank               int64
Date          datetime64[ns]
Price                float64
dtype: object

In [5]:
missing_values.head(10)

RegionID           0
RegionName         0
City               0
State              0
Metro         276395
CountyName         0
SizeRank           0
Date               0
Price         156891
dtype: int64

Here's an overview of the reshaped data:

- ***Data Structure:*** Each row now contains the RegionID, RegionName (zip code), City, State, Metro, CountyName, SizeRank, the Date of the record, and the corresponding Price.

- ***Data Types:*** The Date column has been converted to a datetime type, which is essential for time series analysis. Other columns are appropriately typed (numerical or object).

- ***Missing Values:*** There are missing values in the Metro and Price columns. The missing Metro values may not significantly impact the analysis, as we have other location identifiers like City, State, and CountyName. However, the missing Price values are crucial and need to be addressed.

Next steps in data preprocessing:

- ***Handling Missing Values in Price:*** We need to decide how to handle these missing values. Options include imputation (if the missingness is random and not extensive), or exclusion of records with missing prices. The choice depends on the extent and nature of the missing data.

- ***Exploratory Data Analysis (EDA):*** Before diving into modeling, an exploratory analysis to understand the trends and characteristics of the data is crucial. This will include analyzing price trends over time, price distributions across different regions, and any other relevant factors.

- ***Feature Engineering:*** Based on the EDA, we might identify additional features that could be useful for the analysis, such as indicators for economic cycles, seasonality effects, or regional economic indicators.

- ***Model Selection and Forecasting:*** Once the data is preprocessed, we can select appropriate time series forecasting models to predict future real estate prices.

- ***Evaluation and Selection of Top Zip Codes:*** Using the model's forecasts and possibly other economic indicators, we'll evaluate and rank the zip codes based on their potential for investment.

Given the nature of the Zillow dataset and the objective of forecasting real estate prices to identify the best zip codes for investment, the approach to handling missing values and the decision on whether to format the data for time series analysis first are interconnected. Here's an outline of our approach:

1. **Understanding Missing Values in Price Data**:
   - **Nature of Missingness**: Determine if the missing values are random or systematic. If the missingness is systematic (e.g., missing for specific time periods or specific regions), this could indicate data collection issues or absence of data for newer markets.
   - **Percentage of Missingness**: Assess the proportion of missing data. A high percentage of missing data in certain zip codes might lead to unreliable forecasts for those areas.

2. **Approaches to Handle Missing Values**:
   - **Imputation**: If the missingness is random and not extensive, imputation techniques can be used. Advanced machine learning techniques like K-Nearest Neighbors (KNN) or time series specific methods (like linear interpolation or seasonal decomposition) can be applied.
   - **Exclusion**: If the missingness is extensive or systematic, it might be better to exclude those zip codes from the analysis to avoid introducing bias.

3. **Preparing for Time Series Analysis**:
   - **Time Series Formatting**:This includes setting the date as an index and ensuring that the data is sorted chronologically.
   - **Handling Missing Dates**: If there are missing dates (time points) in the series, they should be identified. Techniques like forward-filling, backward-filling, or interpolation can be used depending on the nature of the data.

4. **Exploratory Data Analysis (EDA)**:
   - Before delving into modeling, conducting EDA is crucial to understand the underlying patterns, trends, and anomalies in the data.



In [6]:
# Analyzing the extent of missing Price values and their nature
missing_price_data = zillow_long[zillow_long['Price'].isna()]

# Checking if missing values are random or systematic
# 1. Checking the distribution of missing values over time
missing_over_time = missing_price_data['Date'].dt.year.value_counts().sort_index()

# 2. Checking the distribution of missing values across different zip codes
missing_by_zip = missing_price_data['RegionName'].value_counts()

missing_over_time_summary = missing_over_time.describe()
missing_by_zip_summary = missing_by_zip.describe()

missing_over_time_summary


count       19.000000
mean      8257.421053
std       4138.164023
min        336.000000
25%       5702.000000
50%       9170.000000
75%      12432.000000
max      12462.000000
Name: count, dtype: float64

In [7]:
missing_by_zip_summary

count    1039.000000
mean      151.001925
std        43.712452
min        15.000000
25%       111.000000
50%       167.000000
75%       183.000000
max       219.000000
Name: count, dtype: float64

In [8]:
missing_over_time.head()

Date
1996     9351
1997    12462
1998    12432
1999    12432
2000    12432
Name: count, dtype: int64

In [9]:
missing_by_zip.head()

RegionName
35759    219
62870    219
48157    219
62215    219
19954    219
Name: count, dtype: int64



The analysis of missing values in the `Price` column reveals the following:

1. **Missing Values Over Time**:
   - The missing values are spread across 19 years (1996-2014).
   - There's a high variation in the number of missing values per year, with some years having significantly more missing values than others. This indicates that missingness might be systematic to certain time periods.

2. **Missing Values Across Zip Codes**:
   - The missing values are spread across 1,039 different zip codes.
   - There's also variation in the number of missing values per zip code. Some zip codes have more missing data, indicating that certain areas might have more incomplete records.

Given these observations, the approach to handle missing values could be as follows:

- **Imputation for Random Missingness**: For zip codes with a relatively low number of missing values, imputation might be a suitable approach. Techniques like linear interpolation or time series specific methods (like seasonal decomposition) can be used, as they can account for the temporal nature of the data.

- **Exclusion for Systematic Missingness**: For years or zip codes with a high number of missing values, it might be better to exclude those records. This is especially true for zip codes with missing values across a significant portion of the time series, as imputation in such cases might introduce bias.

Before proceeding with imputation or exclusion, it's also essential to format the data correctly for time series analysis:

- **Time Series Formatting**: Ensure each time series (each zip code) is in chronological order and set the date as an index. This will facilitate further time series specific processing and analysis.

- **Handling Missing Dates**: If there are entire missing dates (time points) in the series, decide on a strategy to handle these, such as forward-filling, backward-filling, or interpolation.



In [10]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Selecting zip codes with a moderate number of missing values for imputation
# Setting a threshold for maximum missing values allowed for a zip code to be considered for imputation
max_missing_threshold = 50  

# Filtering out zip codes with missing values above the threshold
zip_codes_for_imputation = missing_by_zip[missing_by_zip <= max_missing_threshold].index
data_for_imputation = zillow_long[zillow_long['RegionName'].isin(zip_codes_for_imputation)]

# Preparing data for KNN imputation
# Dropping non-numeric columns and setting the date as index
data_for_imputation_numeric = data_for_imputation.drop(columns=['RegionID', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'])
data_for_imputation_numeric.set_index('Date', inplace=True)

# Standardizing the data before imputation (KNN imputer is sensitive to the scale of the data)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_for_imputation_numeric)

# Applying KNN imputation
knn_imputer = KNNImputer(n_neighbors=5)  # The number of neighbors can be adjusted
data_imputed = knn_imputer.fit_transform(data_scaled)

# Inversing the scaling to get the original scale of prices back
data_imputed_original_scale = scaler.inverse_transform(data_imputed)

# Creating a DataFrame from the imputed data
imputed_data_df = pd.DataFrame(data_imputed_original_scale, columns=data_for_imputation_numeric.columns, index=data_for_imputation_numeric.index)

# Checking the first few rows of the imputed data
imputed_data_df.head()


,RegionName,Price
Date,,
1996-04-01,23192.0,251640.0
1996-04-01,23015.0,183580.0
1996-04-01,23047.0,267760.0
1996-05-01,23192.0,251640.0
1996-05-01,23015.0,183580.0


## ***Next steps:***

- ***Integrating Imputed Data Back:*** The imputed data needs to be integrated back into the main dataset. We'll replace the original missing values in these selected zip codes with the imputed values.

- ***Handling Zip Codes with Extensive Missingness:*** For zip codes with a high number of missing values (above the threshold), we'll exclude them from the analysis to avoid introducing bias.

- ***Time Series Formatting:***
  - Ensure the data for each zip code is in chronological order.
  - Handle any missing dates in the time series, if necessary, using techniques like forward-filling, backward-filling, or interpolation.

- ***Exploratory Data Analysis (EDA):*** Before modeling, it's essential to perform EDA to understand the patterns and trends in the data, which will inform the choice of forecasting models.

- ***Model Selection and Forecasting:*** Select appropriate time series forecasting models for the analysis.

In [16]:
# Integrating the imputed data back into the main dataset
# First, we separate the data that did not require imputation (either no missing values or too many missing values)
data_no_imputation_needed = zillow_long[~zillow_long['RegionName'].isin(zip_codes_for_imputation)]

# Combining the imputed data with the rest of the data
combined_data = pd.concat([data_no_imputation_needed, imputed_data_df.reset_index()])

# Sorting the data by RegionName and Date to ensure proper time series format
combined_data.sort_values(by=['RegionName', 'Date'], inplace=True)

# Handling Zip Codes with Extensive Missingness
# Filtering out zip codes with missing values above the set threshold
zip_codes_to_exclude = missing_by_zip[missing_by_zip > max_missing_threshold].index
final_data = combined_data[~combined_data['RegionName'].isin(zip_codes_to_exclude)]

# Checking for any remaining missing values
remaining_missing_values = final_data.isnull().sum()

# Ensuring proper time series formatting
# Checking if each zip code time series is complete and in chronological order
time_series_completeness = final_data.groupby('RegionName').apply(lambda x: x['Date'].is_monotonic_increasing and len(x['Date']) == len(pd.date_range(start=x['Date'].min(), end=x['Date'].max(), freq='M')))

# Summary of the final prepared dataset for time series analysis
final_data = final_data.head(20)
time_series_completeness_summary = time_series_completeness.value_counts()



In [12]:

final_data

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,Date,Price
5850,58196.0,1001.0,Agawam,MA,Springfield,Hampden,5851.0,1996-04-01,113100.0
20573,58196.0,1001.0,Agawam,MA,Springfield,Hampden,5851.0,1996-05-01,112800.0
35296,58196.0,1001.0,Agawam,MA,Springfield,Hampden,5851.0,1996-06-01,112600.0
50019,58196.0,1001.0,Agawam,MA,Springfield,Hampden,5851.0,1996-07-01,112300.0
64742,58196.0,1001.0,Agawam,MA,Springfield,Hampden,5851.0,1996-08-01,112100.0


In [13]:
remaining_missing_values

RegionID         795
RegionName         0
City             795
State            795
Metro         209880
CountyName       795
SizeRank         795
Date               0
Price              0
dtype: int64

In [14]:
time_series_completeness_summary

False    13687
Name: count, dtype: int64